In [1]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mlp
import matplotlib.pyplot as plt
import seaborn as sns
import time
import re, pip, conda

# 二 贝叶斯优化的实现

贝叶斯优化是当今黑盒函数估计领域最为先进和经典的方法，在同一套序贯模型下使用不同的代理模型以及采集函数、还可以发展出更多更先进的贝叶斯优化改进版算法，因此，贝叶斯优化的其算法本身就多如繁星，实现各种不同种类的贝叶斯优化的库也是琳琅满目，几乎任意一个专业用于超参数优化的工具库都会包含贝叶斯优化的内容。我们可以在以下页面找到大量可以实现贝叶斯优化方法的HPO库：https://www.automl.org/automl/hpo-packages/ ，其中大部分库都是由独立团队开发和维护，因此不同的库之间之间的优劣、性格、功能都有很大的差异。在课程中，我们将介绍如下三个可以实现贝叶斯优化的库：`bayesian-optimization`，`hyperopt`，`optuna`。

|HPO库|优劣评价|推荐指数|
|-|-|-|
|**bayes_opt**|✅实现基于高斯过程的贝叶斯优化<br>✅当参数空间由大量连续型参数构成时<br><br>⛔包含大量离散型参数时避免使用<br>⛔算力/时间稀缺时避免使用|⭐⭐|
|**hyperopt**|✅实现基于TPE的贝叶斯优化<br>✅支持各类提效工具<br>✅进度条清晰，展示美观，较少怪异警告或报错<br>✅可推广/拓展至深度学习领域<br><br>⛔不支持基于高斯过程的贝叶斯优化<br>⛔代码限制多、较为复杂，灵活性较差|⭐⭐⭐⭐|
|**optuna**|✅（可能需结合其他库）实现基于各类算法的贝叶斯优化<br>✅代码最简洁，同时具备一定的灵活性<br>✅可推广/拓展至深度学习领域<br><br>⛔非关键性功能维护不佳，有怪异警告与报错|⭐⭐⭐⭐|

注意，以上三个库<font color="red">**都不支持基于Python环境的并行或加速**</font>，大多数优化算法库只能够支持基于数据库（如MangoDB，mySQL）的并行或加速，但以上库都可以被部署在分布式计算平台。

三个库极其辅助包的安装方法分别如下，使用pip或conda安装时注意关闭梯子。

- Bayes_opt

In [ ]:
#!pip install bayesian-optimization
#!conda install -c conda-forge bayesian-optimization

- Hyperopt

In [ ]:
#!pip install hyperopt

- Optuna

In [ ]:
#!pip install optuna
#!conda install -c conda-forge optuna

- Skopt（作为Optuna辅助包安装，也可单独使用）

In [ ]:
#!pip install scikit-optimize

接下来我们会分别使用三个库来实现贝叶斯优化。在课程中，我们依然使用集成算法中的房价数据作为验证数据，并且呈现出我们之前在不同优化方法上得出的结果作为对比。同时，我们将使用与集成算法中完全一致的随机数种子、以及随机森林算法作为被优化的评估器。

- **导入库，确认使用数据**

In [4]:
#基本工具
import numpy as np
import pandas as pd
import time
import os #修改环境设置

#算法/损失/评估指标等
import sklearn
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.model_selection import KFold, cross_validate

#优化器
from bayes_opt import BayesianOptimization

import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss

import optuna

Bayes_opt版本：1.2.0

In [5]:
print(optuna.__version__)

2.10.0


In [6]:
print(hyperopt.__version__)

0.2.7


In [7]:
data = pd.read_csv(r"D:\myJupyter\机器学习\datasets\House Price\train_encode.csv",index_col=0)

X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [8]:
X.head()

,Id,住宅类型,住宅区域,街道接触面积(英尺),住宅面积,街道路面状况,巷子路面状况,住宅形状(大概),住宅现状,水电气,...,半开放式门廊面积,泳池面积,泳池质量,篱笆质量,其他配置,其他配置的价值,销售月份,销售年份,销售类型,销售状态
0,0.0,5.0,3.0,36.0,327.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,8.0,4.0
1,1.0,0.0,3.0,51.0,498.0,1.0,0.0,3.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,8.0,4.0
2,2.0,5.0,3.0,39.0,702.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2.0,8.0,4.0
3,3.0,6.0,3.0,31.0,489.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8.0,0.0
4,4.0,5.0,3.0,55.0,925.0,1.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2.0,8.0,4.0


In [9]:
X.shape

(1460, 80)

- **确认该数据集上的历史成果**

|HPO方法|默认参数|网格搜索|随机搜索|随机搜索<br>(大空间)|随机搜索<br>(连续型)|
|:-:|:-:|:-:|:-:|:-:|:-:|
|搜索空间/全域空间|-|1536/1536|800/1536|1536/3000|1536/无限|
|运行时间（分钟）|-|6.36|<font color="green">**2.83(↓)**</font>|<font color="green">**3.86(↓)**</font>|3.92|
|搜索最优（RMSE）|30571.266|29179.698|29251.284|<font color="green">**29012.905(↓)**</font>|29148.381|
|重建最优（RMSE）|-|28572.070|<font color="brown">**28639.969(↑)**</font>|<font color="green">**28346.673(↓)**</font>|28495.682|

## 2 基于HyperOpt实现TPE优化

Hyperopt优化器是目前最为通用的贝叶斯优化器之一，Hyperopt中集成了包括随机搜索、模拟退火和TPE（Tree-structured Parzen Estimator Approach）等多种优化算法。相比于Bayes_opt，Hyperopt的是更先进、更现代、维护更好的优化器，也是我们最常用来实现TPE方法的优化器。在实际使用中，相比基于高斯过程的贝叶斯优化，基于高斯混合模型的TPE在大多数情况下以更高效率获得更优结果，该方法目前也被广泛应用于AutoML领域中。TPE算法原理可以参阅原论文[Multiobjective tree-structured parzen estimator for computationally expensive optimization problems](https://www.researchgate.net/publication/342537251_Multiobjective_tree-structured_parzen_estimator_for_computationally_expensive_optimization_problems)，在这里我们将重点介绍关于Hyperopt中使用TPE进行超参数搜索的过程。

In [2]:
import hyperopt
from hyperopt import hp, fmin, tpe, Trials, partial
from hyperopt.early_stop import no_progress_loss

In [3]:
print(hyperopt.__version__)

0.2.7


- 1 定义目标函数

在定义目标函数$f(x)$时，我们需要严格遵守需要使用的当下优化库的基本规则。与Bayes_opt一样，Hyperopt也有一些特定的规则会限制我们的定义方式，主要包括：

> 1 **目标函数的输入必须是符合hyperopt规定的字典**，不能是类似于sklearn的参数空间字典、不能是参数本身，更不能是数据、算法等超参数以外的元素。因此在自定义目标函数时，我们需要让超参数空间字典作为目标函数的输入。<br><br>
> 2 **Hyperopt只支持寻找$f(x)$的最小值，不支持寻找最大值**，因此当我们定义的目标函数是某种正面的评估指标时（如准确率，auc），我们需要对该评估指标取负。如果我们定义的目标函数是负损失，也需要对负损失取绝对值。当且仅当我们定义的目标函数是普通损失时，我们才不需要改变输出。

In [10]:
def hyperopt_objective(params):
    
    #定义评估器
    #需要搜索的参数需要从输入的字典中索引出来
    #不需要搜索的参数，可以是设置好的某个值
    #在需要整数的参数前调整参数类型
    reg = RFR(n_estimators = int(params["n_estimators"])
              ,max_depth = int(params["max_depth"])
              ,max_features = int(params["max_features"])
              ,min_impurity_decrease = params["min_impurity_decrease"]
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1)
    
    #交叉验证结果，输出负根均方误差（-RMSE）
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,verbose=False
                                     ,n_jobs=-1
                                     ,error_score='raise'
                                    )
    
    #最终输出结果，由于只能取最小值，所以必须对（-RMSE）求绝对值
    #以求解最小RMSE所对应的参数组合
    return np.mean(abs(validation_loss["test_score"]))

- 2 定义参数空间

在任意超参数优化器中，优化器会将参数空格中的超参数组合作为备选组合，一组一组输入到算法中进行训练。在贝叶斯优化中，超参数组合会被输入我们定义好的目标函数$f(x)$中。

在hyperopt中，我们使用特殊的字典形式来定义参数空间，其中键值对上的键可以任意设置，只要与目标函数中索引参数的键一致即可，键值对的值则是hyperopt独有的hp函数，包括了：

> **hp.quniform("参数名称", 下界, 上界, 步长)** - 适用于均匀分布的浮点数<br><br>
> **hp.uniform("参数名称",下界, 上界)** - 适用于随机分布的浮点数<br><br>
> **hp.randint("参数名称",上界)** - 适用于[0,上界)的整数，区间为前闭后开<br><br>
> **hp.choice("参数名称",["字符串1","字符串2",...])** - 适用于字符串类型，最优参数由索引表示<br><br>
> **hp.choice("参数名称",[\*range(下界，上界，步长)])** - 适用于整数型，最优参数由索引表示<br><br>
> **hp.choice("参数名称",[整数1,整数2,整数3,...])** - 适用于整数型，最优参数由索引表示<br><br>
> **hp.choice("参数名称",["字符串1",整数1,...])** - 适用于字符与整数混合，最优参数由索引表示

在hyperopt的说明当中，并未明确参数取值范围空间的开闭，根据实验，如无特殊说明，hp中的参数空间定义方法应当都为前闭后开区间。我们依然使用在随机森林上获得最高分的随机搜索的参数空间：

In [11]:
param_grid_simple = {'n_estimators': hp.quniform("n_estimators",80,100,1)
                     , 'max_depth': hp.quniform("max_depth",10,25,1)
                     , "max_features": hp.quniform("max_features",10,20,1)
                     , "min_impurity_decrease":hp.quniform("min_impurity_decrease",0,5,1)
                    }

由于hp.choice最终会返回最优参数的索引，容易与数值型参数的具体值混淆，而hp.randint又只能够支持从0开始进行计数，因此我们常常会使用quniform获得均匀分布的浮点数来替代整数。对于需要取整数的参数值，如果采用quniform方式构筑参数空间，则需要在目标函数中使用int函数限定输入类型。例如，在范围[0,5]中取值时，可以取出[0.0, 1.0, 2.0, 3.0,...]这种均匀浮点数，在输入目标函数时，则必须确保参数值前存在int函数。当然，如果使用hp.choice则不会存在该问题。

由于不涉及到连续型变量，因此我们可以计算出当前参数空间的大小：

In [12]:
len([*range(80,100,1)])*len([*range(10,25,1)])*len([*range(10,20,1)])*len([range(0,5,1)])

3000

- 3 定义优化目标函数的具体流程

有了目标函数和参数空间，接下来我们就可以进行优化了。在Hyperopt中，我们用于优化的基础功能叫做fmin，在fmin中，我们可以自定义使用的代理模型（参数`algo`），一般来说我们有`tpe.suggest`以及`rand.suggest`两种选项，前者指代TPE方法，后者指代随机网格搜索方法。我们还可以通过partial功能来修改算法涉及到的具体参数，包括模型具体使用了多少个初始观测值（参数`n_start_jobs`），以及在计算采集函数值时究竟考虑多少个样本（参数`n_EI_candidates`）。当然，我们也可以不填写这些参数，就使用默认的参数值。

除此之外，Hyperopt当中还有两个值得注意的功能，一个记录整个迭代过程的`trials`，另一个是提前停止参数`early_stop_fn`。其中，`trials`直译为“实验”或“测试”，表示我们不断尝试的每一种参数组合，这个参数中我们一般输入从hyperopt库中导入的方法Trials()，当优化完成之后，我们可以从保存好的trials中查看损失、参数等各种中间信息；而提前停止参数`early_stop_fn`中我们一般输入从hyperopt库导入的方法no_progress_loss()，这个方法中可以输入具体的数字n，表示当损失连续n次没有下降时，让算法提前停止。由于贝叶斯方法的随机性较高，当样本量不足时需要多次迭代才能够找到最优解，因此一般no_progress_loss()中的数值不会设置得太高。在我们的课程中，由于数据量较少，我设置了一个较高的值来避免迭代停止太早。

In [13]:
def param_hyperopt(max_evals=100):
    
    #保存迭代过程
    trials = Trials()
    
    #设置提前停止
    early_stop_fn = no_progress_loss(100)
    
    #定义代理模型
    #algo = partial(tpe.suggest, n_startup_jobs=20, n_EI_candidates=50)#自定义代理模型,这里定为tpe,设置初始观测数量为20,采集函数的备选值是50,参数空间越大设的越大
    params_best = fmin(hyperopt_objective #目标函数
                       , space = param_grid_simple #参数空间
                       , algo = tpe.suggest #代理模型你要哪个呢？,这里使用tpe默认值
                       #, algo = algo
                       , max_evals = max_evals #允许的迭代次数
                       , verbose=True
                       , trials = trials
                       , early_stop_fn = early_stop_fn
                      )
    
    #打印最优参数，fmin会自动打印最佳分数
    print("\n","\n","best params: ", params_best,
          "\n")
    return params_best, trials

- 4 定义验证函数（非必要）

In [14]:
def hyperopt_validation(params):    
    reg = RFR(n_estimators = int(params["n_estimators"])
              ,max_depth = int(params["max_depth"])
              ,max_features = int(params["max_features"])
              ,min_impurity_decrease = params["min_impurity_decrease"]
              ,random_state=1412
              ,verbose=False
              ,n_jobs=-1
             )
    cv = KFold(n_splits=5,shuffle=True,random_state=1412)
    validation_loss = cross_validate(reg,X,y
                                     ,scoring="neg_root_mean_squared_error"
                                     ,cv=cv
                                     ,verbose=False
                                     ,n_jobs=-1
                                    )
    return np.mean(abs(validation_loss["test_score"]))

- 5 执行实际优化流程

In [17]:
params_best, trials = param_hyperopt(30) #1%的空间大小

100%|█████████████████████████████████████████████████| 30/30 [00:06<00:00,  4.86trial/s, best loss: 28553.15406747252]

 
 best params:  {'max_depth': 23.0, 'max_features': 14.0, 'min_impurity_decrease': 1.0, 'n_estimators': 94.0} 



In [64]:
params_best, trials = param_hyperopt(100) #3%的空间大小

100%|███████████████████████████████████████████████| 100/100 [00:21<00:00,  4.71trial/s, best loss: 28450.06487530331]

 
 best params:  {'max_depth': 22.0, 'max_features': 14.0, 'min_impurity_decrease': 0.0, 'n_estimators': 94.0} 



In [50]:
params_best, trials = param_hyperopt(300) #10%的空间大小

 92%|██████████████████████████████████████████▍   | 277/300 [01:01<00:05,  4.52trial/s, best loss: 28346.672687223065]

 
 best params:  {'max_depth': 22.0, 'max_features': 14.0, 'min_impurity_decrease': 0.0, 'n_estimators': 89.0} 



In [33]:
hyperopt_validation(params_best)

28346.672687223065

In [18]:
#打印所有搜索相关的记录
trials.trials[0]

{'state': 2,
 'tid': 0,
 'spec': None,
 'result': {'loss': 28766.452192638408, 'status': 'ok'},
 'misc': {'tid': 0,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'max_depth': [0],
   'max_features': [0],
   'min_impurity_decrease': [0],
   'n_estimators': [0]},
  'vals': {'max_depth': [13.0],
   'max_features': [18.0],
   'min_impurity_decrease': [4.0],
   'n_estimators': [80.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2021, 12, 24, 13, 33, 19, 633000),
 'refresh_time': datetime.datetime(2021, 12, 24, 13, 33, 19, 840000)}

In [20]:
#打印全部搜索的目标函数值
trials.losses()[:10]

[28766.452192638408,
 29762.22885008687,
 29233.57333898302,
 29257.33343872428,
 29180.63733732971,
 29249.676793746046,
 29309.41793204717,
 28915.33638544984,
 29122.269575607537,
 29150.39720576636]

|HPO方法|默认参数|网格搜索|随机搜索|随机搜索<br>(大空间)|随机搜索<br>(连续型)|贝叶斯优化<br>(基于GP)|贝叶斯优化<br>(基于TPE)|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|搜索空间/全域空间|-|1536/1536|800/1536|1536/3000|1536/无限|300/无限|277/3000|
|运行时间（分钟）|-|6.36|<font color="green">**2.83(↓)**</font>|<font color="green">**3.86(↓)**</font>|3.92|<font color="green">**2.11(↓)**</font>|<font color="green">**1.00(↓)**</font>|
|搜索最优（RMSE）|30571.266|29179.698|29251.284|<font color="green">**29012.905(↓)**</font>|29148.381|<font color="green">**28346.673(↓)**</font>|<font color="green">**28346.673(-)**</font>|
|重建最优（RMSE）|-|28572.070|<font color="brown">**28639.969(↑)**</font>|<font color="green">**28346.673(↓)**</font>|28495.682|<font color="green">**28346.673(-)**</font>|<font color="green">**28346.673(-)**</font>|

由于具有提前停止功能，因此基于TPE的hyperopt优化可能在我们设置的迭代次数被达到之前就停止，也因此hyperopt迭代到实际最优值所需的迭代次数可能更少。同时，TPE方法相比于高斯过程计算会更加迅速，因此在运行277次迭代的情况下，hyperopt只需要1分钟时间，而运行300次迭代的bayes_opt却需要2.11分钟，可见，即便运行同样的迭代次数，hyperopt也是更有优势的，这或许是因为hyperopt的参数空间更加稀疏、在整数型参数搜索上更高效。

不过HyperOpt的缺点也很明显，那就是代码精密度要求较高、灵活性较差，略微的改动就可能让代码疯狂报错难以跑通。同时，HyperOpt所支持的优化算法也不够多，如果我们专注地使用TPE方法，则掌握HyperOpt即可，如果我们希望拥有丰富的HPO手段，则可以更深入地接触Optuna库。